# Collecting Data

In [1]:
## Gross Domestic Product data from 'https://www.kaggle.com/datasets/nitishabharathi/gdp-per-capita-all-countries'
## World Press Freedom index data from 'https://rsf.org/en/index'
## Corruption index data from 'https://www.transparency.org/en/cpi/2022'
## Coalition governments data from Bing's AI

Importing necessary libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Scraping list of countries

In [ ]:
## URL to scrape election dates
url = "https://www.worldometers.info/geography/how-many-countries-are-there-in-the-world/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")
soup1 = BeautifulSoup(soup.prettify())
ex = soup1.find('table',{'id':'example2'}).find('tbody').find_all('tr')

country_list = []
for i in ex:
    dic = {}
    c = i.find_all('td')[1].text
    c = c.strip()
    dic['Country'] = c
    country_list.append(dic)
country_list

Save to csv locally

In [ ]:
df = pd.DataFrame(country_list)
df.to_csv('Countries.csv',index_label='id',quotechar='"')

Scraping elections for each country

In [ ]:
def mod_df(url,headers):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    soup1 = BeautifulSoup(soup.prettify())
    ex = soup1.find('table',{'id':'electionsTable'}).find('tbody').find_all('tr')
    
    table = []
    for i in ex:
        dic = {}
        c1 = i.find_all('td')[1].text
        c1 = c1.strip()
        c2 = i.find_all('td')[2].text
        c2 = c2.strip()
        c3 = i.find_all('td')[3].text
        c3 = c3.strip()
        dic['Country'] = c1
        dic['Election'] = c2
        dic['Date'] = c3
        table.append(dic)
    return table

def clean(df):
    df['Date'] = df['Date'].str.replace("\n","")
    df['Date'] = df['Date'].str.replace("(","")
    df['Date'] = df['Date'].str.replace(")","")
    df['Date'] = df['Date'].str.replace(" t","")
    df['Date'] = df['Date'].str.replace(" d","")
    df['Date'] = df['Date'].apply(pd.to_datetime)
    name = "Elections_Countries.csv"
    df.to_csv(name,index_label='id')

ids = [str(i) for i in range(1,241)]
dfs = []
for i in ids:
    url = "https://www.electionguide.org/elections/type/custom/?country_id="+i+"&election_institution_type_id=&year="
    print(url)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"}
    table = mod_df(url,headers)
    df = pd.DataFrame(table)
    dfs.append(df)
df_all = pd.concat(dfs)
clean(df_all)